In [1]:
from pathlib import Path # reads paths in the current OS
import pandas as pd
import numpy as np
import yaml
import utils as ut

In [2]:
with open(Path('conf') / 'paths.yaml') as file:
    config = yaml.full_load(file)

abastiment = pd.read_csv(Path(config['input_path']) / "abastiment_new.csv", sep=",").fillna('')
pagesos    = pd.read_csv(Path(config['input_path']) / "pagesos_clean.csv", sep=",").fillna('')

In [5]:
pagesos.head()

,URL,PRODUCTOR,MARCA,MUNICIPIO,DONDE,PRODUCTOS,OTROS,PAGO,INFO,VERDURA,...,bread_pastries,pasta,others,iseco,n_main_prod,n_other_prod,n_tot_prod,n_paym_methods,n_comarcas_delivery,comarca_origin
0,https://pagesiaacasa.cat/productor/piteus-vins...,"PITEUS vins, vermuts i cava",PITEUS,Corbera del Llobregat (Baix Llobregat),"Alt Camp, Alt Empordà, Alt Penedès, Alt Urgell...",,Vi,"Efectiu, Transferència prèvia, Targeta",,,...,0,0,0,0,0,1,1,3,42,Baix Llobregat
1,https://pagesiaacasa.cat/productor/jordi-riera...,Jordi Riera Flores,Molí de Rué,Vinebre (Ribera d’Ebre),"Alt Camp, Alt Empordà, Alt Penedès, Alt Urgell...",,"Oli, Vi i caves","Efectiu, Bizum, Transferència prèvia, Targeta",<p>Vins DOP Tarragona</p>,,...,0,0,0,0,0,2,2,4,42,Ribera d'Ebre
2,https://pagesiaacasa.cat/productor/agusti-segr...,Agustí Segranyes i Carulla (Producció ecològica),La Cabana d'Alcover SL,Ponts (Noguera),"Alt Camp, Alt Empordà, Alt Penedès, Alt Urgell...",,Oli,"Efectiu, Transferència prèvia",,,...,0,0,0,0,0,1,1,2,42,Noguera
3,https://pagesiaacasa.cat/productor/pilar-comes,Pilar Comes (Producció ecològica),PARC DE LES OLORS DEL SERRAT,SANTA EULÀLIA DE RONÇANA (Vallès Oriental),"Alt Camp, Alt Empordà, Alt Penedès, Alt Urgell...",,Herbes Aromàtiques,Transferència prèvia,,"Bledes, Calçots, Carxofes, Enciams, Escaroles,...",...,0,0,0,0,1,1,2,1,42,Vallès Oriental
4,https://pagesiaacasa.cat/productor/sergi-marti...,Sergi Martín García (Producció integrada),COLLITA DE L'AVI PIÀ,Valls (Alt Camp),"Alt Camp, Alt Empordà, Alt Penedès, Alt Urgell...",,Oli,"Efectiu, Bizum, Transferència prèvia, Targeta",,"Cebes, Tomàquets",...,0,0,0,0,1,1,2,4,42,Alt Camp


In [6]:
abastiment.rename(columns={'PROJECTE':'MARCA'},inplace=True)
pagesos['dataset']    = 'pagesos'
abastiment['dataset'] = 'abastiment'

In [7]:
matches = pagesos['MARCA'].apply(lambda row: ut.run_project_match(row, abastiment['MARCA']))
matches = matches.str.split(';',expand=True).rename(columns={0:'exact',1:'MARCA'})

Partial match: BdeGust Beer Cervesa Compromesa vs. BdeGust Cervesa Compromesa 91 81
Partial match: BIOGRASSFEED vs. Biograssfed 96 91
Partial match: Tomating vs. Tomàting 88 88
Partial match: Ramaderia Ecològica ca l'Andreu vs. Ramaderia ecológica ca l’andreu 94 94
Partial match: L'Enllaç de l'Ebre vs. L'Enllaç de l’Ebre, SCCL 79 89
Partial match: (Mu)DA Vedella d'Agullana vs. MuDA. Vedella d'Agullana 94 96
Partial match: Granja de Montmelús vs. Granja i formatgeria de Montmelús 73 84
Partial match: Safra del Montsec vs. Safrà del Montsec 94 94
Partial match: Celler Optim vs. Celler Lo 76 89
Partial match: FORMATGERIA LAURA vs. Formatgeria Tiraval 78 82
Partial match: Cordú Fruits vs. Cor de fruita 80 83
Partial match: PLANESES AGRICULTIRA REGENERATIVA vs. Planeses Agricultura Regenerativa 97 97
Partial match: Granja Can Torrent vs. Granja can Borrell 83 83
Partial match: Maduixes del Maresme vs. Maduixes de Munt 72 81
Partial match: Can Terrades Produccio Agraria Ecològica sco vs. Ca

In [8]:
# manually checked for names that where filtered as a match but weren't
not_duplicates = ['Celler Lo','Formatgeria Tiraval','Cor de fruita','Granja can Borrell','Maduixes de Munt']

In [9]:
# keep the values that are duplicates only
matches = matches[matches.exact != '0']

In [10]:
# remove false positives
ind = matches[matches['MARCA'].str.contains('\b'+r'\b|\b'.join(not_duplicates)+'\b')].index
matches = matches.drop(ind)

In [11]:
common_cols = [c for c in pagesos.columns if c in abastiment.columns]

In [12]:
all_data = pd.concat([pagesos[common_cols],
          abastiment[~(abastiment.MARCA.isin(matches.MARCA.values))][common_cols]],
          axis=0)

In [13]:
all_data.to_csv(Path(config['input_path']) / 'all_data.csv', index=False)